In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import sklearn
import random

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
torch.manual_seed(1)
random.seed(1)
if device == "cuda":
    torch.cuda.manual_seed(1)

In [5]:
x_train = pd.read_csv("./data2/x_train.csv",engine='python',encoding='CP949')
y_train = pd.read_csv("./data2/y_train.csv",engine='python',encoding='CP949')
x_test = pd.read_csv("./data2/x_test.csv",engine='python',encoding='CP949')
submission = pd.read_csv("./data2/sample_submission.csv")

In [7]:
print(x_train)
print(y_train)
print(x_test)
print(submission)

       시도명  읍면동명         거주인구        근무인구        방문인구       총 유동인구   평균 속도  \
0        0    26    76018.965    5009.811   36887.341   117916.117  40.467   
1        1    24  1232416.968   76043.955  451558.268  1760019.191  38.126   
2        1    23   134260.946    8412.761   78177.981   220851.688  51.107   
3        1    25   289327.429   39102.424  287029.303   615459.156  33.067   
4        1     7  1116617.660   96560.651  524905.301  1738083.612  40.392   
...    ...   ...          ...         ...         ...          ...     ...   
21481    0     6    91977.598   14802.819   79885.446   186665.863  41.676   
21482    0     9    63673.610    3725.663   65813.385   133212.658  48.737   
21483    1     3   406035.642   46335.550  258079.137   710450.329  49.314   
21484    1    25   315737.894   25379.224  283079.548   624196.666  35.312   
21485    1    27  1092141.759  185878.376  518636.283  1796656.419  33.438   

       평균 소요 시간   평균 기온    일강수량  평균 풍속   월   일     년  
0       

In [8]:
x_train = np.array(x_train)
# y_train = np.array(y_train) # shape (21486,1)
y_train = np.array(y_train['교통량']) # shape (21486)
x_test = np.array(x_test)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [9]:
x_train = torch.Tensor(x_train).to(device)
x_test = torch.Tensor(x_test).to(device)
y_train = torch.Tensor(y_train).to(device)

In [10]:
print(x_train.shape,y_train.shape)

torch.Size([21486, 14]) torch.Size([21486])


In [11]:
layer1 = torch.nn.Linear(14,14,bias=True).to(device)
layer2 = torch.nn.Linear(14,8,bias=True).to(device)
layer3 = torch.nn.Linear(8,8,bias=True).to(device)
layer4 = torch.nn.Linear(8,14,bias=True).to(device)
layer5 = torch.nn.Linear(14,1,bias=True).to(device)
relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(0.3).to(device)

In [12]:
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)
torch.nn.init.xavier_normal_(layer5.weight)

Parameter containing:
tensor([[-0.5216, -0.3971,  0.2030,  0.3714, -0.1270, -0.4978, -0.4808,  0.3438,
          0.2109, -0.1461, -0.1662, -0.2172, -0.8686,  0.0196]],
       device='cuda:0', requires_grad=True)

In [13]:
model = torch.nn.Sequential(layer1,relu,dropout,
                           layer2,relu,dropout,
                           layer3,relu,dropout,
                           layer4,relu,dropout,
                           layer5).to(device)

In [18]:
# epochs 2000, lr 1e-2 -> score 0.474...
# epochs 2000, lr 1e-1 -> score 0.385... overfitting

epochs= 2000
lr = 1e-1
loss = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(),lr=lr)

In [19]:
model.train()
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h = model(x_train)
    cost = loss(h,y_train.unsqueeze(1))
    
    cost.backward()
    optim.step()
    
    if epoch % 100 == 0:
        print(cost.item())

139657.734375
140601.75
130718.125
118194.1953125
110208.5703125
108075.046875
106946.46875
107665.8046875
108255.875
106030.4140625
104095.4609375
106771.0390625
104450.078125
104173.8359375
105732.5859375
104159.1328125
100036.9375
101910.8359375
101260.4609375
100873.4765625
100665.640625


In [17]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
submission['predict'] = predict.cpu().detach().numpy()
print(submission)

          id     predict
0          0   77.298416
1          1  150.496582
2          2  158.196320
3          3  218.039505
4          4  591.838379
...      ...         ...
10578  10578  280.343201
10579  10579   77.298416
10580  10580  313.725525
10581  10581   77.298416
10582  10582  370.808624

[10583 rows x 2 columns]


In [ ]:
submission.to_csv("submission.csv",index=False)